# Classical Simulations: Water Splitting Catalyst Screening
## Complete DFT Workflow with Lithium Catalyst Model

---

## Overview

This notebook demonstrates the **classical computational approach** to catalyst screening for water splitting using:
- **Density Functional Theory (DFT)** for quantum mechanical calculations
- **PBE Functional** for exchange-correlation approximation
- **PySCF** for molecular quantum chemistry

### Key Reactions
- **HER (Cathode)**: 2H⁺ + 2e⁻ → H₂  
- **OER (Anode)**: 2H₂O → O₂ + 4H⁺ + 4e⁻  

### Workflow

1. Define molecular geometries computationally  
2. DFT calculations for ground-state energies  
3. Calculations of adsorption energies  
4. Simple catalytic cycle / volcano interpretation  
5. Extract integrals for quantum computing  
6. TODO: Map integrals to qubits and run VQE  


## Installation & Imports


In [ ]:
%pip install -q pyscf pandas numpy

from pyscf import gto, scf, dft
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

print("✓ All imports successful")
print("✓ Ready to start classical simulations")


# STEP 1: Molecule Definition

We define all molecular species involved in water splitting:

| Species | Role | Bond Length | Spin State |
|---------|------|-------------|------------|
| H₂O | Reactant (substrate) | 0.96 Å | Singlet (0) |
| H₂ | HER product | 0.74 Å | Singlet (0) |
| O₂ | OER product | 1.21 Å | Triplet (2) |
| H | HER intermediate (H*) | - | Doublet (1) |
| O | OER intermediate (O*) | - | Triplet (2) |
| OH | OER intermediate (OH*) | 0.96 Å | Doublet (1) |
| Li₂ | Catalyst (clean) | 2.67 Å | Singlet (0) |
| Li₂-H | Adsorbate complex | - | Doublet (1) |

**Basis set:** STO-3G (minimal; for demo only).  
Real work: 6-31G*, def2-TZVP, etc.


In [ ]:
print("Defining molecular geometries...\n")

molecules = {}

# 1. H2O
molecules['H2O'] = gto.M(
    atom = [['O', (0.0000, 0.0000, 0.1173)],
            ['H', (0.0000, 0.7572, -0.4692)],
            ['H', (0.0000, -0.7572, -0.4692)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 0,
)

# 2. H2
molecules['H2'] = gto.M(
    atom = [['H', (0.0, 0.0, 0.0)],
            ['H', (0.0, 0.0, 0.74)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 0,
)

# 3. O2
molecules['O2'] = gto.M(
    atom = [['O', (0.0, 0.0, 0.0)],
            ['O', (0.0, 0.0, 1.21)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 2,
)

# 4. H atom
molecules['H'] = gto.M(
    atom = [['H', (0, 0, 0)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 1,
)

# 5. O atom
molecules['O'] = gto.M(
    atom = [['O', (0, 0, 0)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 2,
)

# 6. OH radical
molecules['OH'] = gto.M(
    atom = [['O', (0.0, 0.0, 0.0)],
            ['H', (0.0, 0.96, 0.0)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 1,
)

# 7. Li2 catalyst
molecules['Li2'] = gto.M(
    atom = [['Li', (0, 0, 0)],
            ['Li', (0, 0, 2.67)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 0,
)

# 8. Li2-H adsorbate
molecules['Li2-H'] = gto.M(
    atom = [['Li', (0, 0, 0)],
            ['H',  (0, 0, 1.6)],
            ['Li', (0, 0, 2.67)]],
    basis = 'sto-3g',
    charge = 0,
    spin = 1,
)

rows = []
for name, mol in molecules.items():
    rows.append({
        'Species': name,
        'Atoms': mol.natm,
        'Electrons': mol.nelectron,
        'Basis functions': mol.nao_nr(),
    })

df_mol = pd.DataFrame(rows)
df_mol


# STEP 2: Ground-State Energy Calculations (DFT)

We compute ground-state electronic energies with **PBE DFT**.


In [ ]:
energies = {}

def e_ev(E):
    return E * 27.211

print('Running DFT (PBE) calculations...\n')

# H2O
mf = dft.RKS(molecules['H2O']); mf.xc = 'PBE'; mf.verbose = 0
energies['H2O'] = mf.kernel()

# H2
mf = dft.RKS(molecules['H2']); mf.xc = 'PBE'; mf.verbose = 0
energies['H2'] = mf.kernel()

# O2
mf = dft.UKS(molecules['O2']); mf.xc = 'PBE'; mf.verbose = 0
energies['O2'] = mf.kernel()

# H
mf = scf.UHF(molecules['H']); mf.verbose = 0
energies['H'] = mf.kernel()

# O
mf = scf.UHF(molecules['O']); mf.verbose = 0
energies['O'] = mf.kernel()

# OH
mf = scf.UHF(molecules['OH']); mf.verbose = 0
energies['OH'] = mf.kernel()

# Li2
mf = scf.RHF(molecules['Li2']); mf.verbose = 0
energies['Li2'] = mf.kernel()

# Li2-H
mf = scf.UHF(molecules['Li2-H']); mf.verbose = 0
energies['Li2-H'] = mf.kernel()

energy_rows = []
for k in ['H2O','H2','O2','H','O','OH','Li2','Li2-H']:
    E = energies[k]
    energy_rows.append({
        'Species': k,
        'Energy (Hartree)': f'{E: .6f}',
        'Energy (eV)': f'{e_ev(E): .4f}',
    })

df_energies = pd.DataFrame(energy_rows)
df_energies


# STEP 3: Adsorption & Reaction Energies


In [ ]:
# H adsorption on Li2: Li2 + H -> Li2-H
delta_E_H = energies['Li2-H'] - energies['Li2'] - energies['H']

# Overall water splitting: H2O -> H2 + 1/2 O2
delta_E_reaction = energies['H2'] + 0.5 * energies['O2'] - energies['H2O']

print('H Adsorption on Li₂ (HER key intermediate)\n')
print(f'ΔE_H (Hartree) = {delta_E_H: .6f}')
print(f'ΔE_H (eV)      = {e_ev(delta_E_H): .4f}\n')

print('Water Splitting: H₂O → H₂ + ½O₂\n')
print(f'ΔE_reaction (Hartree) = {delta_E_reaction: .6f}')
print(f'ΔE_reaction (eV)      = {e_ev(delta_E_reaction): .4f}')


In [ ]:
print('Assessment of H adsorption (volcano concept):\n')

dEh = e_ev(delta_E_H)

if dEh < 0:
    print(f'• H binds (ΔE_H = {dEh:.3f} eV)')
    if -0.3 < dEh < 0:
        print('• In or near optimal HER range (-0.3, 0) eV')
    elif dEh <= -0.3:
        print('• Binding too strong, H might not desorb easily')
else:
    print(f'• H does NOT bind (ΔE_H = {dEh:.3f} eV) – bad HER catalyst')

print('\nAssessment of water splitting energetics:\n')
dEr = e_ev(delta_E_reaction)
if dEr > 0:
    print(f'• Reaction is ENDOTHERMIC: requires ≈ {dEr:.2f} eV per H₂O')
    print('• This is why external voltage (electrolysis) is needed.')
else:
    print('• Reaction would be exothermic (unphysical for water splitting).')


# STEP 5: Extract Integrals for Quantum Computing

We extract one- and two-electron integrals (h1e, h2e) and overlap matrices S for H₂O, H₂, and Li₂-H.


In [ ]:
integrals = {}

print('Extracting one- and two-electron integrals for key systems...\n')

def extract_integrals(label, mol, method='RKS'):
    if method == 'RKS':
        mf = dft.RKS(mol); mf.xc = 'PBE'; mf.verbose = 0
    elif method == 'UKS':
        mf = dft.UKS(mol); mf.xc = 'PBE'; mf.verbose = 0
    elif method == 'RHF':
        mf = scf.RHF(mol); mf.verbose = 0
    else:
        mf = scf.UHF(mol); mf.verbose = 0
    mf.kernel()
    h1e = mf.get_hcore()
    h2e = mol.intor('int2e')
    S   = mf.get_ovlp()
    integrals[label] = {
        'h1e': h1e,
        'h2e': h2e,
        'S': S,
        'nmo': mol.nao_nr(),
        'nelectron': mol.nelectron,
        'nuclear_rep': mf.energy_nuc(),
    }
    print(label + ':', 'h1e', h1e.shape, ', h2e', h2e.shape, ', S', S.shape, ', nmo=', mol.nao_nr(), ', nelec=', mol.nelectron)

extract_integrals('H2O', molecules['H2O'], method='RKS')
extract_integrals('H2',  molecules['H2'],  method='RKS')
extract_integrals('Li2-H', molecules['Li2-H'], method='UHF')


# TODO: Quantum Computing Phase (Not Implemented Here)

Now that integrals are available in `integrals[...]`, the next steps are:

1. **Build Molecular Hamiltonian**  
2. **Map to Qubits** (Jordan–Wigner or Bravyi–Kitaev)  
3. **Run VQE** to obtain improved energies  
4. **Compare** VQE vs DFT energies for catalyst screening  
